In [51]:
import os

In [52]:
%pwd

'd:\\Chicken-Disease-Classification'

In [62]:
desired_path = r'd:\\Chicken-Disease-Classification\\End-to-End-Chicken-Disease-Classification-using-Fecal-Image'

In [63]:
os.chdir(desired_path)

In [64]:
%pwd

'd:\\Chicken-Disease-Classification\\End-to-End-Chicken-Disease-Classification-using-Fecal-Image'

In [65]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [66]:
import sys
import os

# Add the src directory to the system path
sys.path.append(os.path.join(os.getcwd(), 'src'))

In [67]:
import os
print(os.getcwd())

d:\Chicken-Disease-Classification\End-to-End-Chicken-Disease-Classification-using-Fecal-Image


In [68]:
%pip install pyyaml
%pip install joblib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [69]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [70]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callback_config

In [71]:
import os
import urllib.request as requests
from zipfile import ZipFile
import tensorflow as tf
import time

In [74]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        # Convert the Path object to a string
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),  # Convert Path to string
            save_best_only=True,
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [75]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e


[2024-10-20 18:24:47,344: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-20 18:24:47,350: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-20 18:24:47,352: INFO: common: created directory at: artifacts]
[2024-10-20 18:24:47,355: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-10-20 18:24:47,358: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_root_log]


In [76]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2024-10-20-18-26-17'